# Image classification: cats & dogs

In [ ]:
# Handle imports up-front
import glob
import random
import itertools
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.preprocessing import image


## 1. Data preparation

### 1.1. Load the data paths

In [ ]:
# Set the path to the training data
training_data_path='../train/data/train'

# Get a list of training dog and cat images
training_dogs=glob.glob(f'{training_data_path}/dog/dog.*')
training_cats=glob.glob(f'{training_data_path}/cat/cat.*')

### 1.2. Inspect

In [ ]:
fig, axs = plt.subplots(3,2,figsize=(6, 4))

for cat, dog, row in zip(training_cats, training_dogs, axs):
    for animal, ax in zip([cat, dog], row):
        animal=image.load_img(animal)
        animal=image.img_to_array(animal)
        animal/=255.0
        ax.imshow(animal)
        ax.axis('off')

plt.tight_layout()
plt.show()

## 2. EDA

Let's take a look at a few of our images to get a feel for how image data is structured.

### 2.1. Image data

In [ ]:
from tensorflow.keras.utils import load_img, img_to_array

def preprocess_image(filepath, target_size=(224, 224)):
    # Load the image with the target size
    img = load_img(filepath, target_size=target_size)
    # Convert the image to an array
    img_array = img_to_array(img)
    # Normalize the pixel values
    img_array = img_array / 255.0
    return img_array

# Example: Preprocess one cat and one dog image
cat_image = preprocess_image(training_cats[0])
dog_image = preprocess_image(training_dogs[0])

print(f'Cat image shape: {cat_image.shape}')
print(f'Dog image shape: {dog_image.shape}')


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define required variables
training_data_path = '../train/data/train'  # Update this path as per your dataset
img_height = 128  # Height to resize the images
img_width = 128   # Width to resize the images
batch_size = 32   # Batch size for training

# Set up data augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalize pixel values
    rotation_range=40,  # Random rotation
    width_shift_range=0.2,  # Random width shift
    height_shift_range=0.2,  # Random height shift
    shear_range=0.2,  # Shear transformations
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Random horizontal flip
    fill_mode='nearest'  # Fill mode for newly created pixels
)

# Create the training generator
train_generator = datagen.flow_from_directory(
    training_data_path,
    target_size=(img_height, img_width),  # Resize all images to (img_height, img_width)
    batch_size=batch_size,  # Number of images per batch
    class_mode='binary'  # Binary classification (e.g., cats vs dogs)
)


In [ ]:
sample_batch, sample_labels = next(train_generator)
fig, axs = plt.subplots(3, 3, figsize=(8, 8))

for i, ax in enumerate(axs.flat):
    ax.imshow(sample_batch[i])
    ax.axis('off')
plt.tight_layout()
plt.show()


In [ ]:
# Preprocess one image (e.g., a cat image)
sample_image = preprocess_image(training_cats[0])

# Separate the RGB channels
red_channel = sample_image[:, :, 0]
green_channel = sample_image[:, :, 1]
blue_channel = sample_image[:, :, 2]

# Plot histograms for each channel
plt.figure(figsize=(12, 6))

plt.subplot(1, 3, 1)
plt.hist(red_channel.ravel(), bins=256, color='red', alpha=0.7)
plt.title('Red Channel Histogram')
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')

plt.subplot(1, 3, 2)
plt.hist(green_channel.ravel(), bins=256, color='green', alpha=0.7)
plt.title('Green Channel Histogram')
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')

plt.subplot(1, 3, 3)
plt.hist(blue_channel.ravel(), bins=256, color='blue', alpha=0.7)
plt.title('Blue Channel Histogram')
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()


### 2.2. Image dimensions

Let's take a look at a random sample of images from the dataset and see what their dimensions are.

In [ ]:
from tensorflow.keras.utils import load_img

# Initialize lists to store widths and heights
widths = []
heights = []

# Loop over a subset of images (e.g., 200 images)
sample_images = training_cats[:100] + training_dogs[:100]  # Adjust number as needed

for image_path in sample_images:
    img = load_img(image_path)  # Load image
    widths.append(img.width)   # Extract width
    heights.append(img.height) # Extract height

# Plot histograms of widths and heights
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(widths, bins=30, color='blue', alpha=0.7)
plt.title('Histogram of Image Widths')
plt.xlabel('Width (pixels)')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
plt.hist(heights, bins=30, color='green', alpha=0.7)
plt.title('Histogram of Image Heights')
plt.xlabel('Height (pixels)')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()


### 2.3. Image aspect ratios

In [ ]:
from tensorflow.keras.utils import load_img

# Initialize lists to store widths and heights
widths = []
heights = []

# Loop over a subset of images (e.g., 200 images)
sample_images = training_cats[:100] + training_dogs[:100]  # Adjust number as needed

for image_path in sample_images:
    img = load_img(image_path)  # Load image
    widths.append(img.width)   # Extract width
    heights.append(img.height) # Extract height

# Plot histograms of widths and heights
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(widths, bins=30, color='blue', alpha=0.7)
plt.title('Histogram of Image Widths')
plt.xlabel('Width (pixels)')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
plt.hist(heights, bins=30, color='green', alpha=0.7)
plt.title('Histogram of Image Heights')
plt.xlabel('Height (pixels)')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()


## 3. Build the model

### 3.1. Prepare images for streaming

In [ ]:
def make_datasets(training_data_path: str, image_dim: int, batch_size: int=16):

    training_dataset=tf.keras.utils.image_dataset_from_directory(
        training_data_path,
        validation_split=0.2,
        subset='training',
        seed=315,
        image_size=(image_dim, image_dim),
        batch_size=batch_size
    ).repeat()

    validation_dataset=tf.keras.utils.image_dataset_from_directory(
        training_data_path,
        validation_split=0.2,
        subset='validation',
        seed=315,
        image_size=(image_dim, image_dim),
        batch_size=batch_size
    ).repeat()

    return training_dataset, validation_dataset

training_dataset, validation_dataset=make_datasets(training_data_path, 128)

### 3.1. Model definition

In [ ]:
def compile_model(image_dim, learning_rate):

    initializer=tf.keras.initializers.GlorotUniform(seed=315)

    model=Sequential([
        layers.Input((image_dim, image_dim, 3)),
        layers.Rescaling(1./255),
        layers.Conv2D(16, 3, padding='same', activation='relu', kernel_initializer=initializer),
        layers.MaxPooling2D(),
        layers.Conv2D(32, 3, padding='same', activation='relu', kernel_initializer=initializer),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, padding='same', activation='relu', kernel_initializer=initializer),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation='relu', kernel_initializer=initializer),
        layers.Dense(1, activation='sigmoid', kernel_initializer=initializer)
    ])

    optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['binary_accuracy'])

    return model

model=compile_model(128, 0.001)
model.summary()

### 3.2. Model training

In [ ]:
training_results=model.fit(
  training_dataset,
  validation_data=validation_dataset,
  epochs=10,
  steps_per_epoch=5,
  validation_steps=5
)

In [ ]:
# Define the model
model = compile_model(128, 0.001)


In [ ]:
from tensorflow.keras import Sequential, layers, optimizers
import tensorflow as tf

def compile_model(image_dim, learning_rate):
    initializer = tf.keras.initializers.GlorotUniform(seed=315)

    model = Sequential([
        layers.Input((image_dim, image_dim, 3)),  # Input layer for images of size (image_dim, image_dim, 3)
        layers.Rescaling(1.0 / 255),  # Rescale pixel values to [0, 1]
        layers.Conv2D(16, 3, padding='same', activation='relu', kernel_initializer=initializer),  # First Conv layer
        layers.MaxPooling2D(),  # First MaxPool layer
        layers.Conv2D(32, 3, padding='same', activation='relu', kernel_initializer=initializer),  # Second Conv layer
        layers.MaxPooling2D(),  # Second MaxPool layer
        layers.Conv2D(64, 3, padding='same', activation='relu', kernel_initializer=initializer),  # Third Conv layer
        layers.MaxPooling2D(),  # Third MaxPool layer
        layers.Flatten(),  # Flatten feature maps
        layers.Dense(128, activation='relu', kernel_initializer=initializer),  # Fully connected layer
        layers.Dense(1, activation='sigmoid', kernel_initializer=initializer)  # Output layer for binary classification
    ])

    optimizer = optimizers.Adam(learning_rate=learning_rate)  # Optimizer with the given learning rate
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['binary_accuracy'])  # Compile the model

    return model



In [ ]:
# Train the model
training_results = model.fit(
    training_dataset,
    validation_data=validation_dataset,
    epochs=10,
    steps_per_epoch=5,
    validation_steps=5
)

# Access training_results
print(training_results.history)  # View available keys in training history


In [ ]:
# Extract metrics from training_results
history = training_results.history

# Plot training and validation accuracy
plt.figure(figsize=(12, 6))

# Training and Validation Accuracy
plt.subplot(1, 2, 1)
plt.plot(history['binary_accuracy'], label='Training Accuracy')
plt.plot(history['val_binary_accuracy'], label='Validation Accuracy', linestyle='--')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Training and Validation Loss
plt.subplot(1, 2, 2)
plt.plot(history['loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss', linestyle='--')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Binary Cross-Entropy Loss')
plt.legend()

plt.tight_layout()
plt.show()


### 3.3. Model optimization

In [ ]:
# import numpy as np
import pandas as pd
from tensorflow.keras import Sequential, layers, optimizers

# Define a function to create the model
def compile_model(image_dim, learning_rate):
    initializer = tf.keras.initializers.GlorotUniform(seed=315)
    model = Sequential([
        layers.Input((image_dim, image_dim, 3)),
        layers.Rescaling(1.0 / 255),
        layers.Conv2D(16, 3, padding='same', activation='relu', kernel_initializer=initializer),
        layers.MaxPooling2D(),
        layers.Conv2D(32, 3, padding='same', activation='relu', kernel_initializer=initializer),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, padding='same', activation='relu', kernel_initializer=initializer),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation='relu', kernel_initializer=initializer),
        layers.Dense(1, activation='sigmoid', kernel_initializer=initializer)
    ])

    optimizer = optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['binary_accuracy'])
    return model

# Define hyperparameters to test
learning_rates = [0.0005, 0.001, 0.005]
batch_sizes = [16, 32, 64]

# Initialize results dictionary
results = []

# Loop through learning rates and batch sizes
for lr in learning_rates:
    for batch_size in batch_sizes:
        print(f"Testing learning rate: {lr}, batch size: {batch_size}")
        
        # Prepare datasets
        train_dataset, val_dataset = make_datasets(training_data_path, image_dim=128, batch_size=batch_size)
        
        # Compile the model
        model = compile_model(image_dim=128, learning_rate=lr)
        
        # Train the model for a few epochs
        training_results = model.fit(
            train_dataset,
            validation_data=val_dataset,
            epochs=5,  # Train for fewer epochs for quicker experimentation
            steps_per_epoch=5,
            validation_steps=5,
            verbose=0  # Suppress verbose output for clarity
        )
        
        # Record results
        final_train_accuracy = training_results.history['binary_accuracy'][-1]
        final_val_accuracy = training_results.history['val_binary_accuracy'][-1]
        results.append({
            'learning_rate': lr,
            'batch_size': batch_size,
            'train_accuracy': final_train_accuracy,
            'val_accuracy': final_val_accuracy
        })

# Convert results to a DataFrame for better visualization
import pandas as pd
results_df = pd.DataFrame(results)

# Display the results
import ace_tools as tools; tools.display_dataframe_to_user(name="Optimization Results", dataframe=results_df)


In [ ]:
# Best hyperparameters (replace with the actual best values from your optimization results)
best_learning_rate = 0.001  # Replace with the best learning rate
best_batch_size = 32        # Replace with the best batch size

# Prepare datasets with the best batch size
train_dataset, val_dataset = make_datasets(training_data_path, image_dim=128, batch_size=best_batch_size)

# Compile the model with the best learning rate
model = compile_model(image_dim=128, learning_rate=best_learning_rate)

# Train the model for longer
training_results = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,  # Train for more epochs
    steps_per_epoch=50,  # Increase steps per epoch to cover more data
    validation_steps=20,  # Increase validation steps
    verbose=1  # Show detailed training progress
)

# Plot training and validation metrics
history = training_results.history

import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history['binary_accuracy'], label='Training Accuracy')
plt.plot(history['val_binary_accuracy'], label='Validation Accuracy', linestyle='--')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Loss
plt.subplot(1, 2, 2)
plt.plot(history['loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss', linestyle='--')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Binary Cross-Entropy Loss')
plt.legend()

plt.tight_layout()
plt.show()


## 4. Evaluate the model

In [ ]:
import os
import shutil

# Paths
test_data_path = '/workspaces/dltdestryk-gperdrizet-image-classification-project/test1'
organized_test_path = '/workspaces/dltdestryk-gperdrizet-image-classification-project/organized_test'

# Create subdirectories for each class
os.makedirs(os.path.join(organized_test_path, 'cat'), exist_ok=True)
os.makedirs(os.path.join(organized_test_path, 'dog'), exist_ok=True)

# Move files into their respective class folders
for file_name in os.listdir(test_data_path):
    if file_name.startswith('cat'):
        shutil.move(os.path.join(test_data_path, file_name), os.path.join(organized_test_path, 'cat', file_name))
    elif file_name.startswith('dog'):
        shutil.move(os.path.join(test_data_path, file_name), os.path.join(organized_test_path, 'dog', file_name))

print("Test data reorganized successfully.")


In [ ]:
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array

# Load and preprocess test images
test_images = []
test_labels = []  # Optional: Only if you have labels
for file_name in os.listdir(test_data_path):
    img = load_img(os.path.join(test_data_path, file_name), target_size=(128, 128))
    img_array = img_to_array(img) / 255.0
    test_images.append(img_array)

# Convert to numpy array
test_images = np.array(test_images)

# Predict on test images
predictions = model.predict(test_images)

# Print predictions (for binary classification)
print(predictions[:10])  # Example predictions
